#### Notebook for creation of anndata object and adding the images and spatial coordinates to of Xenium add-on panel healthy gut data
- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Date:** 25th March 2024

##### Import packages

In [1]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
from scipy.io import mmread
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
from scipy.sparse import coo_matrix

In [2]:
import tifffile as tf
from PIL import Image, ImageFile

#### Set up working environment

In [3]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.10.5.post1
scanpy      1.9.8
-----
PIL                         10.2.0
anyio                       NA
arrow                       1.3.0
asciitree                   NA
asttokens                   NA
attr                        23.2.0
attrs                       23.2.0
babel                       2.14.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
brotli                      1.1.0
certifi                     2023.11.17
cffi                        1.16.0
charset_normalizer          3.3.2
cloudpickle                 3.0.0
comm                        0.2.1
cycler                      0.12.1
cython_runtime              NA
dask                        2024.1.1
dask_image                  2023.08.1
datashader                  0.16.0
datatree                    0.0.13
dateutil                    2.8.2
debugpy                     1.8.0
decorator                   5.1.1
defusedxml                  0.7.1
docrep                   

#### Data upload

+ Load h5ad object

In [4]:
adata_xenium = sc.read_10x_h5('/../.././../raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cell_feature_matrix.h5')

reading /mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cell_feature_matrix.h5
 (0:00:00)


+ Load image and convert it from tiff to png

In [ ]:
ome_tiff_path = '/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_pallet/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.ome.tif'

In [ ]:
png_path = '/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.png'

img = tf.imread(ome_tiff_path)


if img.dtype != np.uint8:
    img = (255 * (img - img.min()) / (img.ptp())).astype(np.uint8)

Image.fromarray(img).save(png_path)

print(f"Converted {ome_tiff_path} to {png_path}")

+ Upload image

In [5]:
Image.MAX_IMAGE_PIXELS = None
hires = np.asarray(Image.open('/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Xenium_rotated_image.png'))

+ Rotate image

In [ ]:
original_image_path = '/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/Xenium_V1_hColon_Non_diseased_Base_FFPE_he_image.png'
rotated_image_path = '/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Xenium_rotated_image.png'

In [ ]:
!magick convert original_image_path -rotate -90 rotated_image_path

+ Reduce image size

In [ ]:
import io

hires_image = Image.fromarray(hires)

buf = io.BytesIO()
hires_image.save(buf, format="PNG", quality=40) 
buf.seek(0)

# Load the image back from the buffer
compressed_image = Image.open(buf)


In [ ]:
# Specify the path where you want to save the compressed image
compressed_image_path = '/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Xenium_rotated_and_compressed_image.png'  

# Save the compressed image to the specified path
compressed_image.save(compressed_image_path)

+ Upload low resolution image

In [6]:
Image.MAX_IMAGE_PIXELS = None
lowres = np.asarray(Image.open('/mnt/LaCIE/annaM/gut_project/raw_data/Xenium_10X_datasets/Gut_samples/Xenium_rotated_and_compressed_image.png'))

+ Upload coordinates

In [ ]:
coords = pd.read_csv(
        "/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/cells.csv",
        header=0)

In [9]:
# rename index column in adata_xenium to 'cell_id'
adata_xenium.obs.index.name = 'cell_id'

In [10]:
adata_xenium.obs = adata_xenium.obs.merge(coords, on='cell_id', how='left')

In [11]:
adata_xenium.obsm["spatial"] = adata_xenium.obs[["x_centroid", "y_centroid"]].copy().to_numpy()

In [ ]:
nucleus_boundaries = pd.read_csv(
        "/../../../../raw_data/Xenium_10X_datasets/Gut_samples/Non-diseased_pre-designed_and_add-on_panel/outs/nucleus_boundaries.csv",
        header=0)

In [15]:
spatial_key = "spatial"
library_id = "tissue42"
adata_xenium.uns[spatial_key] = {library_id: {}}
adata_xenium.uns[spatial_key][library_id]["images"] = {}
adata_xenium.uns[spatial_key][library_id]["images"] = {"hires": hires}

In [16]:
adata_xenium.uns[spatial_key][library_id]["images"] = {"lowres": lowres}

In [17]:
adata_xenium

AnnData object with n_obs × n_vars = 275822 × 425
    obs: 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

In [20]:
adata_xenium.uns[spatial_key][library_id]

{'images': {'lowres': array([[[237, 237, 239],
          [237, 237, 239],
          [237, 237, 239],
          ...,
          [228, 228, 228],
          [230, 230, 230],
          [233, 233, 233]],
  
         [[236, 236, 238],
          [236, 236, 238],
          [236, 236, 238],
          ...,
          [231, 231, 231],
          [226, 226, 226],
          [230, 230, 230]],
  
         [[237, 238, 242],
          [234, 235, 239],
          [235, 236, 240],
          ...,
          [232, 232, 232],
          [229, 229, 229],
          [229, 229, 229]],
  
         ...,
  
         [[234, 234, 234],
          [238, 238, 238],
          [239, 239, 239],
          ...,
          [231, 231, 233],
          [232, 232, 234],
          [231, 231, 233]],
  
         [[236, 236, 236],
          [238, 238, 238],
          [238, 238, 238],
          ...,
          [233, 233, 235],
          [232, 232, 234],
          [231, 231, 231]],
  
         [[238, 238, 238],
          [238, 238, 238],
    

In [21]:
adata_xenium.write_h5ad('/../../../../Processed_data/Gut_data/10X_Xenium_data/10X_Xenium_add-on_with_image.h5ad')